In [ ]:
import os
import torch
import torch.nn as nn
from PIL import Image

# Generator Model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = 128 // 4
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.l1(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

def train_and_generate_images(model_path, output_root, class_i, num_folds=5, num_images=100, latent_size=100, start_sf=0):
    """
    Loads the model, iterates over the fold structure, generates images, and saves them.

    Args:
        model_path (str): Path to the model weights file.
        output_root (str): Root directory where images will be saved.
        num_folds (int): Number of folds for which images will be generated.
        num_images (int): Number of images to generate for each fold.
        latent_size (int): Size of the latent vector.
        start_sf (int): Starting number for image naming.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = Generator()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Fold yapısı iterasyonu
    for fold in range(1, num_folds + 1):
        fold_dir = os.path.join(output_root, f"fold_{fold}", "train", f"{class_i}")
        os.makedirs(fold_dir, exist_ok=True)

        print(f"Fold {fold}: Images are being saved to {fold_dir}...")

        with torch.no_grad():
            for i in range(num_images):
                test_input = torch.randn(1, latent_size, device=device)
                prediction = model(test_input).squeeze(0)
                image_array = (prediction.cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5) * 255
                image = Image.fromarray(image_array.astype('uint8'))

                image_path = os.path.join(fold_dir, f'generated_image_{start_sf + i + 1}.jpg')
                image.save(image_path)

        start_sf += num_images
        print(f"Fold {fold} tamamlandı.")

        

In [ ]:
if __name__ == "__main__":
    """
    Hushem
    """
    className = "<Class Name>" #such as 01_Normal, 02_Tapered, 03_Pyriform, 04_Amorphous
    
    modelName = "<Model Name>" #such as LB-EGAN, DRAGAN, etc.
        
    model_path = f'<Model Path>'
    output_root = f'<Output Path>'
    train_and_generate_images(model_path=model_path, output_root=output_root, class_i=f"{className}", 
                              num_folds=5, num_images=500, latent_size=100, start_sf=1001)

Fold 1: Görseller /cta/users/ismail_emir/Desktop/AggreGAN_Test/HUSHEM/Ensemble/10x/10x_500/fold_1/train/04_Amorphous klasörüne kaydediliyor...
Fold 1 tamamlandı.
Fold 2: Görseller /cta/users/ismail_emir/Desktop/AggreGAN_Test/HUSHEM/Ensemble/10x/10x_500/fold_2/train/04_Amorphous klasörüne kaydediliyor...
Fold 2 tamamlandı.
Fold 3: Görseller /cta/users/ismail_emir/Desktop/AggreGAN_Test/HUSHEM/Ensemble/10x/10x_500/fold_3/train/04_Amorphous klasörüne kaydediliyor...
Fold 3 tamamlandı.
Fold 4: Görseller /cta/users/ismail_emir/Desktop/AggreGAN_Test/HUSHEM/Ensemble/10x/10x_500/fold_4/train/04_Amorphous klasörüne kaydediliyor...
Fold 4 tamamlandı.
Fold 5: Görseller /cta/users/ismail_emir/Desktop/AggreGAN_Test/HUSHEM/Ensemble/10x/10x_500/fold_5/train/04_Amorphous klasörüne kaydediliyor...
Fold 5 tamamlandı.
